In [1]:
import tensorflow as tf
import numpy as np
import sys
sys.path.append("/home/zoker/quict")


from QuICT.simulation.state_vector import ConstantStateVectorSimulator
from QuICT.core import Circuit
from QuICT.core.gate import *

2023-03-13 16:37:25.653373: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-13 16:37:26.619335: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /user/local/cuda-11.7/lib64:
2023-03-13 16:37:26.619397: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /user/local/cuda-11.7/lib64:
2023-03-13 16:37:26.619402: W tensorflow/compiler/tf2tensorrt/utils/py_u

In [2]:
def f(params):
    print(params)
    params_np = params.numpy()
    print(params_np)
    
    cir = Circuit(2)
    sim = ConstantStateVectorSimulator()
    H | cir
    Rx(params_np[0]) | cir(0)
    sv = sim.run(cir)
    
    loss = sum(sv).real
    loss = tf.convert_to_tensor(loss.get())

    return loss

param = np.array([3.0])
param = tf.convert_to_tensor(param)
f(param)

tf.Tensor([3.], shape=(1,), dtype=float64)
[3.]


<tf.Tensor: shape=(), dtype=float64, numpy=0.14147440333540579>

In [3]:
def vmap(f):
    def wrapper(*args, **kws):
        def pf(x):
            return f(x, *args[1:], **kws)

        return tf.vectorized_map(pf, args[0])
    return wrapper

In [4]:
def vvag(f):
    vf = vmap(f)
    def wrapper(*args, **kws):
        with tf.GradientTape() as tape:
            x = args[0]
            tape.watch(x)
            vs = vf(*args, **kws)
        grad = tape.gradient(vs, x)
        return vs, grad

    return wrapper

In [5]:
vvag_f = vvag(f)

param = np.array([3.0])
param = tf.convert_to_tensor(param)

opt = tf.keras.optimizers.Adam()
# op = opt.minimize

loss, grad = vvag_f(param)
print(loss)
print(grad)

Tensor("loop_body/GatherV2:0", shape=(), dtype=float64)


AttributeError: in user code:

    File "/tmp/ipykernel_23593/3913751171.py", line 4, in pf
        return f(x, *args[1:], **kws)
    File "/tmp/ipykernel_23593/1307783320.py", line 3, in f
        params_np = params.numpy()

    AttributeError: 'Tensor' object has no attribute 'numpy'
